## Chapter 1.5: Creating models from scratch
***
So far we have shown the basics to create geological models by importing data from an external source (in especial GeoModeller 3D). In this chapter, we will explore the option available in GemPy to create the data directly in GemPy or to modify existing one. In this respect we will delve into the pandas DataFrames that contain the necessary data.

Let's start as always by importing the necessary dependencies:

In [1]:
# These two lines are necessary only if gempy is not installed
import sys, os
sys.path.append("../..")

# Importing gempy
import gempy as gp

# Embedding matplotlib figures into the notebooks
#%matplotlib inline


# Aux imports
import numpy as np
import pandas as pn
import matplotlib
import theano
import qgrid

#%matplotlib widget


In [2]:
#gp.save_model(geo_model)

In [3]:
#

if False:
    geo_model = gp.load_model('CreateModelTuto5.pickle')
else:
    geo_model = gp.create_model('CreateModelTuto5')
    gp.init_data(geo_model, [0, 1000, 0, 1000, -1000, 0], [50, 50, 50])
    geo_model.set_default_surfaces()
    geo_model.add_surface_points(400, 300, -500, 'surface1')
    geo_model.add_surface_points(600, 300, -500, 'surface1')

- check pickle works
- when we set an is fault change the BottomRelation

In [4]:
geo_model = gp.create_model('CreateModelTuto5')
gp.init_data(geo_model, [0, 1000, 0, 1000, -1000, 0], [50, 50, 50])

CreateModelTuto5  2019-03-27 17:20

In [5]:
geo_model.surfaces

,surface,series,order_surfaces,isBasement,id


In [6]:
geo_model.set_default_surfaces()


,surface,series,order_surfaces,isBasement,id
0,surface1,Default series,1,False,1
1,basement,Default series,2,True,2


Some default values but to make the model a bit faster but they are not necessary:

In [7]:
# #geo_model.add_formations(['surface1', 'basement'])
# geo_model.set_default_formations()
# geo_model.add_interfaces(400, 300, -500, 'surface1')
# geo_model.add_interfaces(600, 300, -500, 'surface1')
# #geo_model.set_default_orientation()

In [8]:
gp.set_interpolation_data(geo_model,  verbose=[])

Compiling theano function...
Compilation Done!


,values
dtype,float64
output,geology
theano_optimizer,fast_compile
device,cpu
verbosity,[]


In [9]:
gp.activate_interactive_df(geo_model)

It is important to get df with get to update the models sinde the `activate_interactive` method is called

In [10]:
geo_model.qi.get('orientations')

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [11]:
geo_model.qi.get('surface_points')

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [12]:
geo_model.qi.get('surfaces')

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [13]:
geo_model.qi.get('series')

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [14]:
geo_model.qi.get('faults')

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [15]:
geo_model.qi.get('faults_relations')

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [29]:
geo_model.faults.df.dtypes

isFault     bool
isFinite    bool
dtype: object

In [27]:
geo_model.surfaces.df.dtypes

surface             object
series            category
order_surfaces       int64
isBasement            bool
id                   int64
dtype: object

In [26]:
geo_model.surfaces.df.iloc[0,0]

'surface1'

In [17]:
geo_model.qi.qgrid_kr


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [18]:
geo_model.surface_points.df.dtypes

X                float64
Y                float64
Z                float64
X_r              float64
Y_r              float64
Z_r              float64
surface         category
series          category
id                 int64
order_series       int32
dtype: object

In [19]:
geo_model.update_to_interpolator()
gp.compute_model(geo_model, compute_mesh=False)

AssertionError: To compute the model is necessary at least 2 interface points per layer

In [ ]:
gp.plot.plot_section(geo_model, 15, plot_data=True)

In [ ]:
geo_model.solutions.

In [ ]:
geo_model.solutions.scalar_field_lith

In [ ]:
geo_model.solutions.scalar_field_at_interfaces.sum(axis=0)

In [ ]:
geo_model.additional_data.structure_data.df